In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import integrated_gradients as ig
import tensorflow as tf
print(tf.version.GIT_VERSION, tf.version.VERSION)


def one_hot_encode(df, col='utr', seq_len=50):
    nuc_d = {'a':[1,0,0,0],'c':[0,1,0,0],'g':[0,0,1,0],'t':[0,0,0,1], 'n':[0,0,0,0]}
    vectors=np.empty([len(df),seq_len,4])
    
    for i,seq in enumerate(df[col].str[:seq_len]): 
        seq = seq.lower()
        a = np.array([nuc_d[x] for x in seq])
        vectors[i] = a
    return vectors

unknown 2.7.0


In [2]:
data = pd.read_csv('/Users/frederickkorbel/Documents/projects/paper/data/MRL_pred.csv', index_col = False)
mrl_model = tf.keras.models.load_model('/Users/frederickkorbel/Documents/projects/paper/data/seelig/Supplementary Code/modeling/saved_models/main_MRL_model.hdf5')

seq = one_hot_encode(data, seq_len = 50)
ind = data.index
dummy = seq[1]

#igres = ig.integrated_gradients(mrl_model)

2022-02-15 13:28:31.217528: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
seq.shape

(20000, 50, 4)

In [4]:
steps = 50
baseline = np.zeros(seq[-1:].shape)

linear_path = np.zeros(tuple([steps] + [t for t in seq[-1:].shape]))
for i in range(steps):
    linear_path[i] = baseline + (seq[-1:] - baseline) * (i*1.0/steps) # baseline + delta * alpha

linear_path = linear_path.squeeze()
linear_path = tf.convert_to_tensor(linear_path, dtype = tf.float32)


#compute gradients
with tf.GradientTape() as tape:
    tape.watch(linear_path)
    values = mrl_model(linear_path) #call the model to obtain predictions for each position
    dF_dx = tape.gradient(values, linear_path)

#integral approximation
grads = (dF_dx[:-1] + dF_dx[1:]) / tf.constant(2.0)
integrated_grads = tf.math.reduce_mean(grads, axis=0)
assert seq[-1:].squeeze().shape == integrated_grads.shape

#scale integrated gradients with respect to input
integrated_gradients = (seq[-1:] - baseline) * integrated_grads

In [10]:
seq[-1:]

array([[[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1.,

In [6]:
integrated_gradients.shape

TensorShape([1, 50, 4])

In [7]:
ex = ig.integrated_gradients(mrl_model, seq[-1:])

In [ ]:
array = np.array([1,2,3])
tensor = tf.convert_to_tensor(array)
tensor.dtype

In [12]:
ex

<tf.Tensor: shape=(1, 50, 4), dtype=float32, numpy=
array([[[-0.        , -0.        ,  0.00725279,  0.        ],
        [-0.        , -0.        , -0.        ,  0.01963388],
        [ 0.        , -0.01941998,  0.        ,  0.        ],
        [ 0.00691829, -0.        ,  0.        ,  0.        ],
        [ 0.00537928,  0.        , -0.        ,  0.        ],
        [ 0.02571478, -0.        , -0.        ,  0.        ],
        [ 0.        , -0.00072909, -0.        ,  0.        ],
        [-0.00644819,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.0075308 ,  0.        ,  0.        ],
        [ 0.        ,  0.        , -0.08172376,  0.        ],
        [ 0.        ,  0.        , -0.04164022,  0.        ],
        [ 0.        , -0.        , -0.05832902,  0.        ],
        [ 0.        , -0.00085586, -0.        ,  0.        ],
        [ 0.        , -0.        ,  0.        ,  0.02106302],
        [ 0.        ,  0.        , -0.04373043,  0.        ],
        [ 0.      